# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 12** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Integrantes**:
- Lorena Ruelas Gaytán
- Yael Alejandro Rodríguez Barreto
- Ximena Isaac Horta
- Alberto Renteria Camacho

In [1]:
import findspark
findspark.init()

#### Spark Conexion


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-Recommender-Systems") \
    .master("spark://9c456350d25c:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

#### Data preparation

In [ ]:
from team_name.spark_utils import SparkUtils

data_path = "/home/jovyan/notebooks/data/movie_scores/sample_movielens_ratings.txt"

# Define schema for the DataFrame
schema = SparkUtils.generate_schema([
    ("userId", "integer"),
    ("movieId", "integer"),
    ("rating","integer"),
    ("date","timestamp")
])

rating_df = spark.read \
        .schema(schema) \
        .option("header", "false") \
        .option("delimiter", "::") \
        .csv(data_path)

rating_df = rating_df.drop("date")
rating_df.show(10)
rating_df.printSchema()

#### Configure ALS model

In [ ]:
from pyspark.ml.recommendation import ALS

als = ALS(
    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating", 
    maxIter=100, 
    regParam=0.1, 
    rank=5, # Controls the dimensionality of the latent vector space for 
            # users and items.
    coldStartStrategy="drop"  # Avoids NaN predictions
)

#### Training

In [ ]:
model_als = als.fit(rating_df)

#### Predictions

In [ ]:
user_recommendations = model_als.recommendForAllUsers(numItems=5)
user_recommendations.show(truncate=False)

#### Predictions for all data

In [ ]:
predictions = model_als.transform(rating_df)
predictions.show(truncate=False)

#### Evaluate model

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
# Set up evaluator to compute RMSE
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="rating", 
    predictionCol="prediction"
)

# Calculate RMSE
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error (RMSE) = {rmse}")

In [ ]:
sc.stop()